In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.0 MB/s eta 0:00:00


In [25]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [3]:
fashion_mist = keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels) = fashion_mist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [5]:
test_images[0].shape

(28, 28)

In [8]:
test_images = test_images/255.0
train_images = train_images/255.0

In [12]:
test_images = test_images.reshape(len(test_images),28,28,1)
train_images = train_images.reshape(len(train_images),28,28,1)

In [13]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [18]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(filters=hp.Int('conv_1_filters',min_value=32,max_value=128,step=16),
                          kernel_size=hp.Choice('conv_1_kernels',values=[3,5]),
                          activation='relu',
                          input_shape=(28,28,1)),
      keras.layers.Conv2D(filters=hp.Int('conv_2_filters',min_value=32,max_value=64,step=16),
                          kernel_size=hp.Choice('conv_2_kernels',values=[3,5]),
                          activation='relu'),
      keras.layers.Flatten(),
      keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
  return model

In [19]:
tuner = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='fashion_mnist')

In [20]:
tuner.search(train_images,train_labels,epochs=10,validation_split=0.2)

Trial 5 Complete [00h 01m 15s]
val_accuracy: 0.8673333525657654

Best val_accuracy So Far: 0.8715833425521851
Total elapsed time: 00h 07m 28s


In [21]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        51232     
                                                                 
 flatten (Flatten)           (None, 15488)             0         
                                                                 
 dense (Dense)               (None, 64)                991296    
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 1043818 (3.98 MB)
Trainable params: 1043818 (3.98 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
model.fit(train_images, train_labels, epochs=20, validation_split=0.2, initial_epoch=10)

Epoch 11/20
1500/1500 [==============================] - 8s 6ms/step - loss: 0.3164 - accuracy: 0.8848 - val_loss: 0.3443 - val_accuracy: 0.8742
Epoch 12/20
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3073 - accuracy: 0.8865 - val_loss: 0.3446 - val_accuracy: 0.8737
Epoch 13/20
1500/1500 [==============================] - 7s 5ms/step - loss: 0.2981 - accuracy: 0.8894 - val_loss: 0.3191 - val_accuracy: 0.8802
Epoch 14/20
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2879 - accuracy: 0.8926 - val_loss: 0.3287 - val_accuracy: 0.8802
Epoch 15/20
1500/1500 [==============================] - 7s 5ms/step - loss: 0.2807 - accuracy: 0.8949 - val_loss: 0.3124 - val_accuracy: 0.8849
Epoch 16/20
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2744 - accuracy: 0.8967 - val_loss: 0.3294 - val_accuracy: 0.8786
Epoch 17/20
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2671 - accuracy: 0.9013 - val_loss: 0.3003 - val_ac